# 1. Importing Required Libraries

In [ ]:
# Importing libraries
%pip install pandas numpy matplotlib seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# Display settings
%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 6)



In [ ]:
import yfinance as yf
import os

# Define the stock ticker and time period
ticker = "GOOG"
start_date = "2020-01-01"
end_date = "2025-01-01"

# Fetch the historical stock data
goog_data = yf.download(ticker, start=start_date, end=end_date)

# Define the folder path and file name
folder_path = "data/input"
os.makedirs(folder_path, exist_ok=True)  # Ensure the folder exists
csv_file_name = os.path.join(folder_path, "GOOG_stock_data.csv")

# Save the data to the specified folder
goog_data.to_csv(csv_file_name)

print(f"Data saved to {csv_file_name}")


# 2. Loading Data

In [ ]:
# Load data (adjust file_path to the location of your CSV file)
file_path = "data/input/GOOG_stock_data.csv"
date_column = "Date"
price_column = "Close"

# Read and preprocess data
data = pd.read_csv(file_path)
data[date_column] = pd.to_datetime(data[date_column])
data.set_index(date_column, inplace=True)
data = data.rename(columns={price_column: "close"})

# Display first few rows
print(data.head())


# 3. Summary Statistics

In [ ]:
# Summary statistics for the data
print("Summary Statistics:")
print(data.describe())

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing Values:\n", missing_values)


# 4. Visualizing the Closing Price

In [ ]:
# Line plot of closing prices
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["close"], label="Closing Price", color="blue")
plt.title("Time Series of Closing Prices")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()


# 5. Rolling Statistics

In [ ]:
# Add rolling means
data["short_ma"] = data["close"].rolling(window=20).mean()
data["long_ma"] = data["close"].rolling(window=50).mean()

# Plot rolling means
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["close"], label="Closing Price", alpha=0.6)
plt.plot(data.index, data["short_ma"], label="20-Day MA", color="orange")
plt.plot(data.index, data["long_ma"], label="50-Day MA", color="red")
plt.title("Closing Price with Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.show()


# 6. Daily Returns

In [ ]:
# Calculate daily returns
data["daily_returns"] = data["close"].pct_change()

# Plot histogram of daily returns
plt.figure(figsize=(12, 6))
sns.histplot(data["daily_returns"].dropna(), kde=True, bins=50, color="green")
plt.title("Distribution of Daily Returns")
plt.xlabel("Daily Returns")
plt.ylabel("Frequency")
plt.show()


# 7. Correlation Analysis

In [ ]:
# Correlation heatmap (if additional columns like volume or other features exist)
correlation_matrix = data.corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()


# 8. Maximum Drawdown Analysis

In [ ]:
# Calculate cumulative returns and drawdowns
data["cumulative_returns"] = (1 + data["daily_returns"]).cumprod()
data["cumulative_max"] = data["cumulative_returns"].cummax()
data["drawdown"] = (data["cumulative_returns"] - data["cumulative_max"]) / data["cumulative_max"]

# Plot drawdowns
plt.figure(figsize=(12, 6))
plt.plot(data.index, data["drawdown"], label="Drawdown", color="red")
plt.fill_between(data.index, data["drawdown"], 0, color="red", alpha=0.3)
plt.title("Drawdown Over Time")
plt.xlabel("Date")
plt.ylabel("Drawdown")
plt.legend()
plt.show()


# Create the backtest_result csv

In [ ]:
import yfinance as yf
import pandas as pd
import os

# Download historical data for backtesting
ticker = "GOOG"
start_date = "2020-01-01"
end_date = "2025-01-01"
data = yf.download(ticker, start=start_date, end=end_date)

# Strategy: Moving Average Crossover
short_window = 20  # Short moving average (e.g., 20 days)
long_window = 50   # Long moving average (e.g., 50 days)

# Calculate moving averages
data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
data['Long_MA'] = data['Close'].rolling(window=long_window).mean()

# Generate trading signals
data['Signal'] = 0
data.loc[data['Short_MA'] > data['Long_MA'], 'Signal'] = 1  # Buy signal
data.loc[data['Short_MA'] <= data['Long_MA'], 'Signal'] = -1  # Sell signal

# Simulate trades
data['Daily_Return'] = data['Close'].pct_change()
data['Strategy_Return'] = data['Signal'].shift(1) * data['Daily_Return']
cumulative_strategy_returns = (1 + data['Strategy_Return']).cumprod()

# Calculate summary statistics
total_return = cumulative_strategy_returns.iloc[-1] - 1
win_rate = (data['Strategy_Return'] > 0).sum() / len(data['Strategy_Return'].dropna())
total_trades = len(data['Signal'].dropna())

# Save results to CSV
folder_path = "data/input"
os.makedirs(folder_path, exist_ok=True)  # Ensure directory exists
csv_file_name = os.path.join(folder_path, "backtest_results.csv")

# Create DataFrame for results
results = pd.DataFrame({
    "Metric": ["Total Return", "Win Rate", "Total Trades"],
    "Value": [total_return, win_rate, total_trades]
})

results.to_csv(csv_file_name, index=False)
print(f"Backtest results saved to {csv_file_name}")


# 9. Save Preprocessed Data

In [ ]:
# Save the enriched dataset with rolling statistics, daily returns, etc.
output_path = "data/output/preprocessed_data.csv"
data.to_csv(output_path)
print(f"Preprocessed data saved to {output_path}.")
